# Incorporating custom evaluation criteria for LLM outputs and calibrating TLM trustworthiness scores against human ratings

Are you finding TLM trustworthiness scores do not align with your team's manual quality ratings of good/bad LLM outputs? If so, you can address this via:
- Adjusting prompts and [TLM options/configurations](/tlm/tutorials/tlm_advanced/)
- Custom evaluation criteria
- Calibrating TLM trustworthiness scores against your human quality ratings for example prompt-response pairs.

This tutorial demonstrates how to evaluate and improve the performance of TLM for your specific use-case.

## Setup

This tutorial requires a TLM API key. Get one [here](https://tlm.cleanlab.ai/).

Cleanlab's TLM Python client can be installed using pip. This tutorial also requires the `scikit-learn` package.

In [ ]:
%pip install --upgrade cleanlab-tlm scikit-learn

In [ ]:
# Set your API key
import os
os.environ["CLEANLAB_TLM_API_KEY"] = "<API key>" # Get your free API key from: https://tlm.cleanlab.ai/

In [1]:
import pandas as pd
from scipy.stats import spearmanr  # just used for evaluation of results in this tutorial

pd.set_option('display.max_colwidth', None)  

In [2]:
from cleanlab_tlm import TLM

tlm = TLM()  # See Advanced Tutorial for optional TLM configurations to get better/faster results

## Fetch the dataset

Let's first load an example dataset. Here we consider a dataset composed of trivia questions with responses already computed from some LLM, which we want to automatically evaluate with TLM.

In this tutorial we will be using the `tlm.get_trustworthiness_score()` method to evaluate these prompt-response pairs, and explore how we can use various techniques to calibrate TLM's trustworthiness score to align with your specific use case.

In [ ]:
!wget -nc https://cleanlab-public.s3.amazonaws.com/Datasets/triviaqa-conciseness/triviaqa_with_ratings.csv
!wget -nc https://cleanlab-public.s3.amazonaws.com/Datasets/triviaqa-conciseness/triviaqa_without_ratings.csv

In [6]:
data = pd.read_csv("triviaqa_with_ratings.csv")

Suppose we aim to score LLM responses based on both correctness of the response and conciseness (ie. the LLM response should not contain extraneous information). Our dataset additionally contains human ratings regarding the (manually-determined) quality of each response. Even without having such human ratings, you can still use many of the techniques demonstrated in this tutorial.

Here is a sample question where the LLM response is factually correct, however it contains too much irrelevant information, and hence it is marked as incorrect by a human annotator (rating = 0):

In [7]:
print("Question:", data["question"][0], "\n")
print("Response:", data["answer"][0], "\n")
print("Human Rating:", data["human_rating"][0])

Question: Bonar Law is the only Prime Minister not born in the UK. In which country was he born? 

Response: Bonar Law, who briefly served as British Prime Minister in the early 1920s, holds the distinction of being the only Prime Minister born outside the United Kingdom. He was born in Canada, specifically in the province of New Brunswick. This unique aspect of his background adds an interesting layer to his political legacy, as he later rose to lead the UK government during a particularly tumultuous period in British history. 

Human Rating: 0


The LLM response to this other question is both correct and concise, hence it has been manually marked as high-quality (human rating = 1):

In [8]:
print("Question:", data["question"][1], "\n")
print("Response:", data["answer"][1], "\n")
print("Human Rating:", data["human_rating"][1])

Question: Which singer had a big 60s No 1 with Roses Are Red? 

Response: Bobby Vinton 

Human Rating: 1


## Scoring responses with TLM

At first, let's just append a simple prompt template to each question, and then use `get_trustworthiness_score()` to score each response.

In [9]:
def create_prompt(question):
    return f"Answer this question as concisely as possible: {question}"
    
data["prompt"] = data["question"].apply(create_prompt)
print("Sample prompt:")
print(data["prompt"][0])

Sample prompt:
Answer this question as concisely as possible: Bonar Law is the only Prime Minister not born in the UK. In which country was he born?


In [10]:
res = tlm.get_trustworthiness_score(data["prompt"].tolist(), data["answer"].tolist())
res_df = pd.DataFrame(res)

Querying TLM...   0%|          |

Querying TLM... 100%|██████████|


Spearman's rank correlation is a statistical method used to measure the strength of the relationship between two ranked variables. We can use it here to measure how well the trustworthiness scores from TLM reflect the given human ratings.

In this tutorial, the human ratings are binary (only 0 and 1 values representing correct and incorrect responses). However, all methods showcased in this tutorial can be applied to any range of numeric human ratings, such as ratings on a scale of 1 to 5. Spearman's rank correlation is a valid measure of alignment betweeen estimated trustworthiness scores and human ratings, regardless whether they are binary or numeric, as it quantifies how consistently good responses are scored higher than bad ones.

In [11]:
print("Spearman correlation:", spearmanr(res_df["trustworthiness_score"], data["human_rating"]).statistic)

Spearman correlation: 0.5433137542226836


## Prompt engineering for better TLM trustworthiness scoring

One simple way to improve TLM performance is to improve the instructions we give. With clearer instructions on how the LLM should respond, TLM can better score the responses for our specific use-case.

Here for example, we can update TLM's `prompt` to specify that wordy answers are considered incorrect.

In [12]:
def create_improved_prompt(question):
    return f"""Answer the following question as concisely as you can. 
For the answer to be considered correct, it has to be concise. Wordy answers are incorrect.
Question: {question}"""

data["improved_prompt"] = data["question"].apply(create_improved_prompt)
print("Sample improved prompt:")
print(data["improved_prompt"][0])

Sample improved prompt:
Answer the following question as concisely as you can. 
For the answer to be considered correct, it has to be concise. Wordy answers are incorrect.
Question: Bonar Law is the only Prime Minister not born in the UK. In which country was he born?


In [13]:
res_improved = tlm.get_trustworthiness_score(data["improved_prompt"].tolist(), data["answer"].tolist())
res_improved = pd.DataFrame(res_improved)

Querying TLM... 100%|██████████|


We see that with the updated prompt, TLM's trustworthiness score is now better aligned with our human ratings (as measured via their Spearman correlation):

In [14]:
print("Spearman correlation:", spearmanr(res_improved["trustworthiness_score"], data["human_rating"]).statistic)

Spearman correlation: 0.584929275822719


<details><summary> Prompt engineering tips for TLM <b>(click to expand)</b></summary>

When using the TLM method: `get_trustworthiness_score(prompt, response)`, how you specify the `prompt` argument matters.
Pick a `prompt` that if given to a standard LLM would yield a response that you'd generally consider good.

To identify such a `prompt` template, you can run `TLM.prompt(prompt)` over a dataset with various candidate prompt templates (here using TLM with say `quality_preset='base'`).
The best prompt template should be the one for which `TLM.prompt()` yields responses that look best to you.

Use standard prompt engineering techniques such as: specifying the desired response format, whether to be concise or include explanations/reasoning, what to mention vs. not, how to handle edge-cases, background information to better contextualize the task, etc.
Important elements of an effective `prompt` argument for TLM include: specifying precisely what a **correct answer** should contain, and perhaps also giving examples of wrong answers.
</details>

## Custom evaluation criteria

Our improved prompt template did enhance TLM performance. However, evaluating conciseness of the response represents a particular notion of response-quality that may not align with TLM's general trustworthiness scoring (which aims to quantify uncertainty in the LLM). Fortunately, TLM supports *custom evaluation criteria*, allowing you to define specific ways to additionally evaluate response-quality.

We can define our custom evaluation criteria in TLM's `options` dictionary:

In [15]:
custom_eval_criteria_option = {"custom_eval_criteria": [
    {"name": "Conciseness", "criteria": "Determine if the response is concise."}
]}

tlm_custom_eval = TLM(options=custom_eval_criteria_option)

Note that while the `prompt` argument for `TLM.get_trustworthiness_score()` should only specify **what a correct response should look like**, the custom evaluation criteria should specify **how to determine if the response is good** (more tips below).

After defining the additional evaluation criteria, running TLM's `get_trustworthiness_score()` method will return both TLM's standard trustworthiness scores along with a new custom evaluation score based on our criteria.

In [16]:
res_custom_eval = tlm_custom_eval.get_trustworthiness_score(data["improved_prompt"].tolist(), data["answer"].tolist())
res_custom_eval_df = pd.DataFrame(res_custom_eval)
res_custom_eval_df.head()

Querying TLM... 100%|██████████|


,trustworthiness_score,log
0,0.650516,"{'custom_eval_criteria': [{'name': 'Conciseness', 'score': 0.1714374914049258}]}"
1,0.987435,"{'custom_eval_criteria': [{'name': 'Conciseness', 'score': 0.9975124355614093}]}"
2,0.939139,"{'custom_eval_criteria': [{'name': 'Conciseness', 'score': 0.25124689196087185}]}"
3,0.189058,"{'custom_eval_criteria': [{'name': 'Conciseness', 'score': 0.2605290450034299}]}"
4,0.941208,"{'custom_eval_criteria': [{'name': 'Conciseness', 'score': 0.3310500673501733}]}"


The custom evaluation scores will be returned as entries in the `log` object:

In [17]:
res_custom_eval_df["log"].iloc[0]

{'custom_eval_criteria': [{'name': 'Conciseness',
   'score': 0.1714374914049258}]}

In [ ]:
res_custom_eval_df["custom_score"] = res_custom_eval_df.apply(lambda x: x["log"]["custom_eval_criteria"][0]["score"], axis=1)
res_custom_eval_df["custom_score"].head()

0    0.171437
1    0.997512
2    0.251247
3    0.260529
4    0.331050
Name: custom_score, dtype: float64

Let's combine the original data with these results to see how well the our conciseness criteria scored the responses.

In [19]:
custom_eval_combined = pd.concat([data, res_custom_eval_df], axis=1)
custom_eval_sorted = custom_eval_combined.sort_values("custom_score")

Below, we view 2 examples with the lowest conciseness scores across our dataset, and see that the LLM responses are indeed very verbose. We see that in the first example, TLM's trustworthiness score did not actually reflect the verboseness of the response.

This showcases how specifying custom evaluation criteria can help in use-cases where TLM's general trustworthiness scores do not reflect your notion of good-vs-bad quality responses.

In [20]:
custom_eval_sorted.head(2)[["question", "answer", "trustworthiness_score", "custom_score"]]

,question,answer,trustworthiness_score,custom_score
29,What was the name of Michael Jackson's autobiography written in 1988?,"The title of Michael Jackson's autobiography, written and published in 1988, is 'Moonwalk'. In this deeply personal and revealing book, Jackson reflects on his extraordinary life, starting from his early days as a child prodigy with the Jackson 5 to his unprecedented rise as a global pop sensation. The book delves into his experiences with fame, his artistic inspirations, and the personal challenges he faced throughout his career, offering readers an intimate glimpse into the mind of the King of Pop.",0.664936,0.002911
14,For which movie did Katharine Hepburn win her second Oscar?,"Katharine Hepburn won her second Academy Award for her role in the 1967 film 'Guess Who's Coming to Dinner'. In this acclaimed film, she portrayed Christina Drayton, a mother grappling with her daughter's decision to marry a Black man. Hepburn’s poignant and emotionally charged performance earned her critical praise and solidified her place as one of Hollywood's all-time great actresses. The film, which addresses issues of race and social change, was significant in its time, and Hepburn's portrayal is a key element of its impact.",0.711409,0.003500


Inspecting responses with the highest conciseness scores in our dataset, we see that these are indeed nice and concise.

In [21]:
custom_eval_sorted.tail(2)[["question", "answer", "trustworthiness_score", "custom_score"]]

,question,answer,trustworthiness_score,custom_score
32,"In the run up to the 1997 election, Tony Blair outlined his parties' three priorities for office, can you name them?","Tony Blair's three priorities for office in the run up to the 1997 election were education, education, education.",0.885939,0.997512
34,Marc Dutroux hit the headlines over a 'house of horrors' in which country?,Marc Dutroux hit the headlines over a 'house of horrors' in Belgium.,0.952093,0.997512


TLM's custom evaluation criteria offer tailored **guardrails** to maximize the safety/policy-adherence of your LLM applications.

<details><summary> Tips for writing custom evaluation criteria <b>(click to expand)</b></summary>

You can iterate on how you write out the custom evaluation criteria, in order to better align the custom evaluation scores with what you consider good-vs-bad quality responses.
Tips include: define clear/objective criteria for determining answer quality and avoid subjective language. Consider including: examples of good vs bad responses, and edge-case guidelines -- such as whether things like flowery-language or answer-refusal are considered good or bad.
Qualitatively describe measurable aspects of the response **without describing numerical scoring mechanisms**, as TLM already has an internal scoring system based on your qualitative description.


**Examples of custom evaluation criteria:**

**In customer support applications:** a custom criteria for *Company XYZ* to evaluate the brand-safety of their AI might include:

> *Determine whether the response portrays Company XYZ in a positive or neutral light. A bad response mentions negative aspects of Company XYZ or their products without caveats, or mentions competitors of Company XYZ. A great response does not contain statements which could be perceived as potentially negative regarding Company XYZ or might negatively impact Company XYZ.*


**In customer support applications:** a custom criteria for *Company XYZ* to evaluate the policy-adherence of their AI might include:

> *Determine whether the response strictly adheres to the following Policy Guidelines. A bad response fails to follow one of the guidelines in the policy. A great response does not contain any policy violations, perhaps because the policy is not particularly relevant to this interaction. &#92;n&#92;n &num;&num; Policy Guidelines &#92;n&#92;n {your_policy_document_here}*


**In conversational chat applications:** you may not want to rely on TLM's trustworthiness score for every AI response, but rather only for verifiable statements that convey information. You can specify a custom evaluation criteria like the following:

> *Determine whether the response is non-propositional, in which case it is great. Otherwise it is a bad response if it conveys any specific information or facts, or otherwise seems like an answer whose accuracy could matter.*

And then you could only consider TLM's trustworthiness score for those responses whose custom evaluation score is low.

**Additional Use Cases for Custom Evaluation Criteria:**

**For financial compliance:** a custom criteria to evaluate regulatory adherence might include:

> *Strictly evaluate if the response follows these financial advice regulations:*
>
> *1. Includes necessary disclaimers*
>
> *2. Avoids specific investment recommendations*
>
> *3. Explains risks alongside potential benefits*
>
> *4. Uses clear, non-misleading language*
>
> *5. Encourages consultation with financial advisors*
>
> *A great response meets criteria in all categories. A mediocre response meets most criteria but lacks completeness. A bad response fails multiple criteria or has serious issues in any category.*

**For code quality evaluation:** a custom criteria to evaluate code safety and maintainability might include:

> *Strictly evaluate if the code follows these quality requirements:*
>
> *1. Error Handling: Validates inputs, handles edge cases, uses specific error messages* 
>
> *2. Documentation: Includes docstring, explains complex logic, documents assumptions*
>
> *3. Security: Prevents common vulnerabilities, validates inputs, follows security best practices*
>
> *4. Maintainability: Uses descriptive names, consistent formatting, modular functions*
>
> *A great response meets criteria in all categories. A mediocre response meets most criteria but lacks completeness. A bad response fails multiple criteria or has serious issues in any category.*

**In educational content:** a custom criteria to evaluate pedagogical effectiveness might include:

> *Strictly evaluate if the response meets these educational criteria:*
>
> *1. Provides clear learning objectives upfront*
>
> *2. Uses structured progression from basic to complex*
>
> *3. Includes concrete examples or analogies*
>
> *4. Anticipates and addresses common misconceptions*
>
> *5. Uses precise terminology with appropriate definitions*
>
> *A great response meets criteria in all categories. A mediocre response meets most criteria but lacks completeness. A bad response fails multiple criteria or has serious issues in any category.*

</details>

## Calibrating TLM scores against human quality ratings

TLM's native trustworthiness scoring and custom evaluation criteria each have their strengths. By combining them, we can benefit from TLM's general trustworthiness score while tailoring specific aspects of the evaluation to our specific criteria. None of the above approaches requires human-rating annotations, we have so far just used those to quantify alignment between the scores and human ratings.

Here we showcase how `TLMCalibrated` can combine and calibrate these values into a score that is explicitly aligned against numeric/binary human-rating (i.e. ground-truth) quality annotations. While we here integrate the custom evaluation and trustworthiness scores into a single calibrated score, you can also run the same procedure to calibrate just one of these scores without the other.

First, we instantiate the `TLMCalibrated` object with the same custom evaluation criteria used previously in the `options` argument.

In [22]:
from cleanlab_tlm import TLMCalibrated
tlm_calibrated = TLMCalibrated(options=custom_eval_criteria_option)

Next, we **fit** this `TLMCalibrated` object to a dataset composed of the previously obtained TLM scores and human quality ratings. This trains the model to produce better-aligned scores over this dataset.

We can directly pass `res_custom_eval` (the list of dictionaries returned from the previous `get_trustworthiness_score()` call) directly into `fit()`.

In [23]:
tlm_calibrated.fit(res_custom_eval, data["human_rating"])

After fitting `TLMCalibrated` to our human-annotated dataset, we can call its `get_trustworthiness_score()` method. This returns not only the previous TLM trustworthiness score and custom conciseness score, but also an additional calibrated score that optimally combines those two scores to align with human quality ratings.

In [24]:
calibrated_res = tlm_calibrated.get_trustworthiness_score(data["improved_prompt"].tolist(), data["answer"].tolist())
calibrated_res_df = pd.DataFrame(calibrated_res)

Querying TLM... 100%|██████████|


In [25]:
calibrated_res_df.head()

,trustworthiness_score,log,calibrated_score
0,0.650516,"{'custom_eval_criteria': [{'name': 'Conciseness', 'score': 0.1714374914049258}]}",0.140863
1,0.987435,"{'custom_eval_criteria': [{'name': 'Conciseness', 'score': 0.9975124355614093}]}",1.000000
2,0.939139,"{'custom_eval_criteria': [{'name': 'Conciseness', 'score': 0.25124689196087185}]}",0.796112
3,0.189058,"{'custom_eval_criteria': [{'name': 'Conciseness', 'score': 0.2605290450034299}]}",0.221944
4,0.941208,"{'custom_eval_criteria': [{'name': 'Conciseness', 'score': 0.3310500673501733}]}",0.850461


We again measure alignment between the calibrated scores and our human ratings via their Spearman correlation, which here is the highest among all scoring methods. Thus whenever you have a moderately-sized set of human quality ratings, we recommend considering `TLMCalibrated` to produce a tailored evaluation score for LLM responses in your use-case.

In [26]:
print("Spearman correlation:", spearmanr(calibrated_res_df["calibrated_score"], data["human_rating"]).statistic)

Spearman correlation: 0.7167231544441708


### Using `TLMCalibrated` to get improved scores on new data

You can use the already-fitted `TLMCalibrated` model to score new responses via the same `get_trustworthiness_score()` method. Here we demonstrate this with some additional data.

In [27]:
new_data = pd.read_csv("triviaqa_without_ratings.csv")
new_data["prompt"] = new_data["question"].apply(create_improved_prompt)

We call `get_trustworthiness_score()` on the new (prompt, response) pairs, just as we did before.

In [28]:
new_data_res = tlm_calibrated.get_trustworthiness_score(new_data["prompt"].tolist(), new_data["answer"].tolist())
new_data_res_df = pd.DataFrame(new_data_res)
new_data_res_df.head(2)

Querying TLM... 100%|██████████|


,trustworthiness_score,log,calibrated_score
0,0.972855,"{'custom_eval_criteria': [{'name': 'Conciseness', 'score': 0.9975124358564608}]}",0.925461
1,0.711186,"{'custom_eval_criteria': [{'name': 'Conciseness', 'score': 0.01743449151079022}]}",0.182863


The returned results contain the TLM trustworthiness score and `log` (like before), but also the `calibrated_score` to use for evaluating these additional responses (in real-time).

In [29]:
new_data_combined = pd.concat([new_data, new_data_res_df], axis=1)
new_data_combined.head(2)[["question", "answer", "calibrated_score"]]

,question,answer,calibrated_score
0,"In 1976, which gymnast scored 7 maximum scores of 10 as she won three gold medals, one silver and one bronze?",Nadia Comaneci,0.925461
1,What is the Milky Way?,"The Milky Way is a barred spiral galaxy, one of the billions of galaxies in the observable universe, and it is our home galaxy. As part of the Local Group of galaxies, it exists alongside other notable galaxies like Andromeda and Triangulum, as well as about 54 smaller satellite galaxies. The Milky Way contains between 100 and 400 billion stars, including our Sun, and the dense band of light seen from Earth at night is caused by the multitude of stars and celestial objects aligned along the galactic plane. It is named after the appearance of this bright band of light, which stretches across the night sky.",0.182863


<details><summary> Tips for further scoring improvements <b>(click to expand)</b></summary>

At this point, you have specified custom evaluation criteria and combined it with TLM's native trustworthiness score to produce calibrated scores via `TLMCalibrated`. To further align these scores against your human-quality ratings, you can tinker with the prompts used in various arguments of `TLMCalibrated`.

Try jointly improving the text used for both the `prompt` argument and the custom evaluation criteria. Iterate through many alternative versions of this text, trying to maximize the resulting Spearman correlation between the resulting calibrated scores from `TLMCalibrated` and human-quality ratings.

Tips on tuning both of these include:

- The base TLM prompt will perform better in general correctness evaluation, however you can still be specific about certain guidelines that it can follow.
- For example in RAG use cases: if the correct response to a question is "The given context does not contain any relevant information to answer the question" (because the retrieved context lacks the necessary details), you should explicitly specify this in the prompt. This ensures that TLM recognizes such responses as correct and does not score them low simply because they do not provide a direct answer.
- The custom evaluation criteria is better suited for domain-specific needs beyond a general correctness check. You can specify these custom evaluation criteria to complement the general trustworthiness scoring that the base TLM provides.


By systematically adjusting both the prompt and evaluation criteria, you can improve `TLMCalibrated` scores to suite your specific use-case.
</details>


<details><summary> Thresholding TLMCalibrated scores to classify responses as good vs. bad <b>(click to expand)</b></summary>

To choose a score threshold that best separates High-vs-Low quality responses, follow these steps:

- `Data Splitting`: Divide your labeled dataset into training and validation sets.
- `Threshold Selection`: Fit TLMCalibrated on the training set. Evaluate various threshold values on the held-out validation set to see which value best separates High-vs-Low quality responses (can compute an evaluation metric like accuracy).
- `Final Training`: Optionally, refit your TLMCalibrated model on all of your labeled data (training + validation) once you have selected the threshold to use when scoring future responses.

**Note:** For brevity, this tutorial evaluated our TLMCalibrated scores without proper data splitting. Use held‑out data in practice to avoid overfitting.

</details>

<details><summary> Saving and loading TLMCalibrated objects <b>(click to expand)</b></summary>

After fitting your `TLMCalibrated` object, you can save it for later use by loading it back in. Here's how to do it properly using the available [helper functions](/tlm/api/python/utils.tlm_calibrated):

```python
from cleanlab_tlm.utils.tlm_calibrated import save_tlm_calibrated_state, load_tlm_calibrated_state

# Save the TLMCalibrated state
save_tlm_calibrated_state(tlm_calibrated, 'tlm_calibrated_state.skops')

# Load the TLMCalibrated state
tlm_calibrated = load_tlm_calibrated_state('tlm_calibrated_state.skops')
```
</details>